# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# File to Load
city_data_to_load = "../Results/clean_city_data.csv"

In [3]:
# Read read the city data and store into Pandas DataFrames
city_data = pd.read_csv(city_data_to_load, encoding="utf-8")
city_data.head()

,City,Hemisphere,Lat,Lng,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Date
0,Pimentel,Southern,-6.84,-79.93,59.85,59.85,88,17,12.77,PE,07/28/2020
1,Krasnokamensk,Northern,50.10,118.04,74.84,74.84,63,95,14.65,RU,07/28/2020
2,Kapaa,Northern,22.08,-159.32,82.40,80.01,69,40,21.92,US,07/28/2020
3,Albany,Northern,42.60,-73.97,82.00,77.00,78,60,5.17,US,07/28/2020
4,Borazjan,Northern,29.27,51.22,93.20,93.20,75,40,18.34,IR,07/28/2020


In [4]:
# Print the clean city count
len(city_data)

532

### Humidity Heatmap
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Create a list containing coordinates
coordinates = city_data[["Lat","Lng"]]

In [6]:
# Convert humidity to float
humid = city_data["Humidity"].astype(float)

In [7]:
# Plot Heatmap
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humid, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Create criteria for the perfect vacation climate where's it's not too hot and not too cold
city_subset = city_data[((city_data["Max Temp"] >= 65) & (city_data["Max Temp"] <= 85)) 
                        & ((city_data["Min Temp"] >= 60) & (city_data["Min Temp"] <= 78))
                        & (city_data["Humidity"] <= 55)  
                        & (city_data["Wind Speed"] <= 10)
                        & (city_data["Cloudiness"] <= 40)]
city_subset

,City,Hemisphere,Lat,Lng,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Date
224,Baiyin,Northern,36.56,104.21,75.20,75.20,38,8,8.95,CN,07/28/2020
305,Iporá,Southern,-16.44,-51.12,66.45,66.45,47,0,1.74,BR,07/28/2020
313,Mil'kovo,Northern,54.72,158.62,76.51,76.51,52,18,2.06,RU,07/28/2020
345,Morros,Southern,-9.45,-46.30,68.54,68.54,55,0,6.67,BR,07/28/2020
377,Alta Floresta,Southern,-9.88,-56.09,70.48,70.48,45,0,3.65,BR,07/28/2020
421,Abu Samrah,Northern,35.30,37.18,77.50,77.50,51,6,3.51,SY,07/28/2020
426,Altamont,Northern,42.21,-121.74,75.00,71.01,40,20,6.93,US,07/28/2020
448,Darhan,Northern,49.49,105.92,72.95,72.95,52,0,5.97,MN,07/28/2020
452,Amarante do Maranhão,Southern,-5.57,-46.74,68.88,68.88,50,0,2.08,BR,07/28/2020
469,Murgab,Northern,37.50,61.97,77.00,77.00,44,0,4.59,TM,07/28/2020


In [9]:
len(city_subset)

11

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Remove any cities with missing data
hotel_df = city_subset.dropna()
hotel_df.reset_index(inplace=True)
del hotel_df['index']

# Add column for Hotel Name
hotel_df['Hotel Name'] = ""
hotel_df

,City,Hemisphere,Lat,Lng,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Date,Hotel Name
0,Baiyin,Northern,36.56,104.21,75.20,75.20,38,8,8.95,CN,07/28/2020,
1,Iporá,Southern,-16.44,-51.12,66.45,66.45,47,0,1.74,BR,07/28/2020,
2,Mil'kovo,Northern,54.72,158.62,76.51,76.51,52,18,2.06,RU,07/28/2020,
3,Morros,Southern,-9.45,-46.30,68.54,68.54,55,0,6.67,BR,07/28/2020,
4,Alta Floresta,Southern,-9.88,-56.09,70.48,70.48,45,0,3.65,BR,07/28/2020,
5,Abu Samrah,Northern,35.30,37.18,77.50,77.50,51,6,3.51,SY,07/28/2020,
6,Altamont,Northern,42.21,-121.74,75.00,71.01,40,20,6.93,US,07/28/2020,
7,Darhan,Northern,49.49,105.92,72.95,72.95,52,0,5.97,MN,07/28/2020,
8,Amarante do Maranhão,Southern,-5.57,-46.74,68.88,68.88,50,0,2.08,BR,07/28/2020,
9,Murgab,Northern,37.50,61.97,77.00,77.00,44,0,4.59,TM,07/28/2020,


In [11]:
len(hotel_df)

11

In [15]:
# params dictionary to update each iteration
params = {
    "radius": 10000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": gkey
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        #hotel_df.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        #hotel_df.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [16]:
hotel_df

,City,Hemisphere,Lat,Lng,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Date,Hotel Name
0,Baiyin,Northern,36.56,104.21,75.20,75.20,38,8,8.95,CN,07/28/2020,Baiyin Hotel
1,Iporá,Southern,-16.44,-51.12,66.45,66.45,47,0,1.74,BR,07/28/2020,Grand Hotel
2,Mil'kovo,Northern,54.72,158.62,76.51,76.51,52,18,2.06,RU,07/28/2020,"Hotel ""Geologist"""
3,Morros,Southern,-9.45,-46.30,68.54,68.54,55,0,6.67,BR,07/28/2020,
4,Alta Floresta,Southern,-9.88,-56.09,70.48,70.48,45,0,3.65,BR,07/28/2020,Hotel Mandino
5,Abu Samrah,Northern,35.30,37.18,77.50,77.50,51,6,3.51,SY,07/28/2020,
6,Altamont,Northern,42.21,-121.74,75.00,71.01,40,20,6.93,US,07/28/2020,Best Western Plus Olympic Inn
7,Darhan,Northern,49.49,105.92,72.95,72.95,52,0,5.97,MN,07/28/2020,Buudai Hotel
8,Amarante do Maranhão,Southern,-5.57,-46.74,68.88,68.88,50,0,2.08,BR,07/28/2020,Hotel Santa Rita
9,Murgab,Northern,37.50,61.97,77.00,77.00,44,0,4.59,TM,07/28/2020,


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…